In [1]:
import warnings
import itertools
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt

from itertools import islice, takewhile
from functools import reduce

from geopy.distance import distance

pd.set_option('display.max_rows', 1000000000)

In [2]:
df = pd.read_csv('data/220-buses.csv', 
        parse_dates=['last_modified'],
        true_values=['t'],
        false_values=['f'])

df['longitude'] /= 3600000
df['latitude'] /= 3600000
df.set_index('last_modified', drop=False, inplace=True)

In [14]:
def trips(df):
    return [df[df['trip_id'] == t] for t in df['trip_id'].unique()]

In [15]:
church_east = (51.89217, -8.55789) # lat-lon
church_west = (51.89221, -8.55923)

def stop_distance(r, stop):
    return distance(stop, (r.latitude, r.longitude)).m

In [21]:
def stop_rows(stop, trip, threshold=100):
    def f(acc, d):
        ds = acc[0]
        stops = acc[1]
        if len(ds) > 3:
            d1, d2, d3 = ds[-2].stop_distance, ds[-1].stop_distance, d
            if (d2 <= threshold
                    and d1 > d2
                    and d2 > d3):
                stops = stops + [d2]
        return (ds + [d], stops)
    ds, stops = reduce(f, trip.itertuples(), ([], []))    
    return stops        
#     ds = {r: stop_distance(r, stop) for r in trip.itertuples()}
#     return list(takewhile(lambda r: ds[r] < 100, sorted(ds, key = lambda r: ds[r])))

In [22]:
ts = trips(df)
[stop_rows(church_east, t) for t in ts]

AttributeError: 'Pandas' object has no attribute 'stop_distance'